In [1]:
import pandas as pd
import numpy as np

In [10]:
tf_transform_output.transformed_feature_spec()

{'atemp': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'count': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'datetime': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'daytype': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'humidity': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'season': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'temp': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'weather': FixedLenFeature(shape=[1], dtype=tf.string, default_value=None),
 'windspeed': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}

In [2]:
df = pd.read_csv("../bike-sharing-demand/train.csv")
serving_sample = pd.read_csv("../bike-sharing-demand/test.csv")

In [4]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [6]:
serving_sample.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [7]:
season_di = {1: "spring", 2 : "summer", 3 : "fall", 4 : "winter" }
weather_di = {1: "clear", 2 : "cloudy", 3 : "light rain", 4 : "heavy rain" }

In [8]:
def replace_with_cat(df):
    df.replace({"season": season_di}, inplace=True)
    df.replace({"weather": weather_di}, inplace=True)  

In [9]:
replace_with_cat(df)
replace_with_cat(serving_sample)

In [10]:
def day_type(x):
    if(x["holiday"] == 1):
        return "holiday" 
    elif(x["workingday"] == 1):
        return "workingday" 
    else:
        return "weekend"
    
df['daytype'] = df.apply(lambda x: day_type(x), axis=1)
serving_sample['daytype'] = serving_sample.apply(lambda x: day_type(x), axis=1)

In [11]:
df.drop(['holiday', 'workingday'], axis=1, inplace=True)
serving_sample.drop(['holiday', 'workingday'], axis=1, inplace=True)

In [12]:
cols = ['datetime', 'season', 'weather', 'daytype', 'temp', 'atemp', 
        'humidity', 'windspeed', 'casual', 'registered', 'count']

df = df[cols]
serving_sample = serving_sample[[ elem for elem in cols if elem not in ['casual', 'registered', 'count']] ]

In [13]:
import os

# define the name of the directory(s) to be created
paths = ["bike-sharing-data"]

for path in paths:
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

Creation of the directory bike-sharing-data failed


In [17]:
train, val, test = np.split(df, [int(.6*len(df)), int(.8*len(df))])

In [18]:
train.head()

,datetime,season,weather,daytype,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,spring,clear,weekend,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,spring,clear,weekend,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,spring,clear,weekend,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,spring,clear,weekend,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,spring,clear,weekend,9.84,14.395,75,0.0,0,1,1


In [19]:
val.head()

,datetime,season,weather,daytype,temp,atemp,humidity,windspeed,casual,registered,count
6531,2012-03-09 09:00:00,spring,cloudy,workingday,13.94,14.395,42,35.0008,17,267,284
6532,2012-03-09 10:00:00,spring,cloudy,workingday,13.94,14.395,36,32.9975,14,145,159
6533,2012-03-09 11:00:00,spring,cloudy,workingday,14.76,16.665,34,23.9994,17,170,187
6534,2012-03-09 12:00:00,spring,clear,workingday,14.76,16.665,34,27.9993,27,174,201
6535,2012-03-09 13:00:00,spring,cloudy,workingday,15.58,19.695,29,26.0027,49,175,224


In [20]:
train.to_csv("../bike-sharing-data/train.csv", index=False)
val.to_csv("../bike-sharing-data/val.csv", index=False)
test.to_csv("../bike-sharing-data/test.csv", index=False)

Copy to gs

In [1]:
!head -n 5 ../bike-sharing-data/train.csv

datetime,season,weather,daytype,temp,atemp,humidity,windspeed,casual,registered,count
2011-01-01 00:00:00,spring,clear,weekend,9.84,14.395,81,0.0,3,13,16
2011-01-01 01:00:00,spring,clear,weekend,9.02,13.635,80,0.0,8,32,40
2011-01-01 02:00:00,spring,clear,weekend,9.02,13.635,80,0.0,5,27,32
2011-01-01 03:00:00,spring,clear,weekend,9.84,14.395,75,0.0,3,10,13


In [22]:
!gsutil cp -r ../bike-sharing-data/* gs://bike-sharing-data/

Copying file://bike-sharing-data/test.csv [Content-Type=text/csv]...
Copying file://bike-sharing-data/train.csv [Content-Type=text/csv]...           
Copying file://bike-sharing-data/val.csv [Content-Type=text/csv]...             
/ [3 files][798.1 KiB/798.1 KiB]                                                
Operation completed over 3 objects/798.1 KiB.                                    
